# Toronto - DS Capstone assignment - 3 (follows from cell 18 after assignment 1&2)

### Importing libraies

In [1]:
import pandas as pd
import requests
import lxml.html as lxht

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Processing information about Toronto provided on the Wikipedia link

In [3]:
tor_raw = requests.get(link)
tor_doc = lxht.fromstring(tor_raw.content)
tor_con = tor_doc.xpath('//tr')

### Creating a List with first element

In [4]:
tor_con = tor_doc.xpath('//tr')

tor_col=[]
n=0

for k in tor_con[0]:
    n+=1
    tor_name=k.text_content()
    print ('%d:"%s"'%(n,tor_name))
    tor_col.append((tor_name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


### Creating a DataFrame

In [5]:
for k in range(1,len(tor_con)):
    tor=tor_con[k]
    if len(tor)!=3:
        break
    
    n=0
    for m in tor.iterchildren():
        tor_data=m.text_content() 
        if n>0:
            try:
                tor_data=int(tor_data)
            except:
                pass
        tor_col[n][1].append(tor_data)
        n+=1
        
tor_dict={title:column for (title,column) in tor_col}
df=pd.DataFrame(tor_dict)

In [6]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


### Cleaning the DataFrame

In [7]:
df = df.replace('\n','', regex=True)
df.columns = ['Postalcode', 'Borough', 'Neighbourhood',]
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Dropping 'Not assigned' cells from the Borough column of the DataFrame

In [8]:
df.drop(df.index[df['Borough'] == 'Not assigned'], axis = 0, inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Grouping the DataFrame by Postalcode and Borough

In [9]:
df = df.groupby(['Postalcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postalcode','Borough','Neighbourhood']
df.head()

,Postalcode,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


### Dropping 'Canadian postal codes' cell from the Borough column of the DataFrame

In [10]:
df.drop(df.index[df['Borough'] == 'Canadian postal codes'], axis = 0, inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Assigning Neighbourhood = Borough where Neighbourhood is 'Not Assigned'

In [11]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [12]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Checking shape as the final step of the assignment

In [13]:
df.shape

(103, 3)

### Creating DataFrame from the location data link as the Geocoder package did not work

In [14]:
location = 'https://cocl.us/Geospatial_data'

df1 = pd.read_csv(location)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two data sets

In [15]:
df1.columns = ['Postalcode','Latitudes','Longitudes']
df1.head()

,Postalcode,Latitudes,Longitudes
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_merge = pd.merge(df, df1, on='Postalcode')
df_merge.head()

,Postalcode,Borough,Neighbourhood,Latitudes,Longitudes
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
df_merge.shape

(103, 5)

### Importing Libraries

In [18]:
import json
from pandas.io.json import json_normalize
import matplotlib.cm as mcm
import matplotlib.colors as mcs
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

### Creating Toronto's map

In [19]:
tor_lat = 43.653963
tor_long = -79.387207
tor_map = folium.Map(location=[tor_lat, tor_long], zoom_start=10)
for lat, long, boro, neigh in zip(df_merge['Latitudes'], df_merge['Longitudes'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neigh, boro)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(tor_map)
tor_map

### Listing Foursquare credentials

In [20]:
Client_id = 'L5CQWSEYKAAVGX5TBPUGCZDHY3W30HSRCTGO4HPDXRVDMORG'
Client_secret = 'W0AJ1JOCNXF0JWCDLWPCPIS5QP4JCYKZXTDZXAX30IJPQ55Y'
Version = '20180605'
limit = 100

### Defining a function to process Toronto's neighbourhood within 1000 metres

In [21]:
def explore_venues(venues, lat, long, rad=1000):
    
    list_locations=[]
    for venue, lt, lng in zip(venues, lat, long):
        print(venue)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_id, 
            Client_secret, 
            Version, 
            lt, 
            lng, 
            rad, 
            limit)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        list_locations.append([(
            venue, 
            lt, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    closeby_venues = pd.DataFrame([item for k in list_locations for item in k])
    closeby_venues.columns = ['Neighbourhood', 
                  'Latitudes', 
                  'Longitudes', 
                  'Venue name', 
                  'Venue lat', 
                  'Venue long', 
                  'Venue category']
    
    return(closeby_venues)

### Creating a neighbourhood database

In [22]:
df_tor_ven = explore_venues (venues =df_merge['Neighbourhood'],
                                   lat =df_merge['Latitudes'],
                                   long =df_merge['Longitudes'],
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

### Viewing the  database

In [23]:
df_tor_ven.head()

,Neighbourhood,Latitudes,Longitudes,Venue name,Venue lat,Venue long,Venue category
0,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
3,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant


In [24]:
df_tor_ven.shape

(4891, 7)

### Grouping the database by Neighbourhood and Analysing it

In [25]:
df_tor_ven.groupby('Neighbourhood').count()

,Latitudes,Longitudes,Venue name,Venue lat,Venue long,Venue category
Neighbourhood,,,,,,
Agincourt,46,46,46,46,46,46
"Alderwood, Long Branch",25,25,25,25,25,25
"Bathurst Manor, Wilson Heights, Downsview North",29,29,29,29,29,29
Bayview Village,13,13,13,13,13,13
"Bedford Park, Lawrence Manor East",42,42,42,42,42,42
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",12,12,12,12,12,12
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",48,48,48,48,48,48


In [36]:
print('There are {} venues.'.format(len(df_tor_ven['Venue category'].unique())))

There are 327 venues.


In [37]:
# Creating new DataFrame with neighbourhoods as index and venues as columns
df_tor_analysis = pd.get_dummies(df_tor_ven[['Venue category']], prefix="", prefix_sep="")
df_tor_analysis['Neighbourhood'] = df_tor_ven['Neighbourhood'] 
neigh_column = [df_tor_analysis.columns[-1]] + list(df_tor_analysis.columns[:-1])
df_tor_analysis = df_tor_analysis[neigh_column]
#Grouping same neighbourhoods together and taking mean for venues (to get back to 98 neighbourhoods)
df_tor_grouped = df_tor_analysis.groupby('Neighbourhood').mean().reset_index()
df_tor_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,...,0.02381,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0


In [38]:
df_tor_grouped.shape

(98, 328)

### Creating a DataFrame with Top 5 venues

In [48]:
# Defining a top 5 function
def top5_function(category, number):
    tor_rows = category.iloc[1:]
    tor_rows_arranged = tor_rows.sort_values(ascending=False)
    return tor_rows_arranged.index.values[0:number]

In [50]:
# Creating a DataFrame with Top 5 venues a columns
import numpy as np
number = 5
neigh = ['Neighbourhood']
for k in np.arange(number):
    try:
        neigh.append('Top Venue Number {}{}'.format(k+1))
    except:
        neigh.append('Top Venue Number {}'.format(k+1))
df_neigh = pd.DataFrame(columns=neigh)
df_neigh['Neighbourhood'] = df_tor_grouped['Neighbourhood']

for k in np.arange(df_tor_grouped.shape[0]):
    df_neigh.iloc[k, 1:] = top5_function(df_tor_grouped.iloc[k, :], number)

df_neigh.head()

,Neighbourhood,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
0,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Bakery,Sandwich Place
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Pharmacy,Sandwich Place,Intersection
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Gas Station,Supermarket
3,Bayview Village,Japanese Restaurant,Bank,Grocery Store,Gas Station,Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Restaurant,Fast Food Restaurant,Sandwich Place


### Clustering neighbourhoods

In [57]:
# Run KMeans clustering
k_neigh = 6
tor_clusters = df_tor_grouped.drop('Neighbourhood', 1)
tor_kmeans = KMeans(n_clusters=k_neigh, random_state=0).fit(tor_clusters)

In [58]:
# Creating a common all inclusive DataFrame
df_neigh.insert(0, 'Labels', tor_kmeans.labels_)
df_tor_final = df_merge
df_tor_final = df_tor_final.join(df_neigh.set_index('Neighbourhood'), on='Neighbourhood')
df_tor_final.head()

ValueError: cannot insert Labels, already exists

### Creating map for visualization

In [75]:
# Visualizing on map
tor_map_clusters = folium.Map(location=[tor_lat, tor_long], zoom_start=10)
z = np.arange(k_neigh)
mps = [k + z + (k*z)**2 for k in range(k_neigh)]
tor_markers = []
for lat, long, pi, clu in zip(df_tor_final['Latitudes'], df_tor_final['Longitudes'], df_tor_final['Neighbourhood'], df_tor_final['Labels']):
    marker_label = folium.Popup(str(pi) + ' Cluster ' + str(clu), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=marker_label,
        fill=True,
        fill_opacity=0.7).add_to(tor_map_clusters)
       
tor_map_clusters

### Analyzing the 6 clusters

In [69]:
df_tor_final.loc[df_tor_final['Labels'] == 0, df_tor_final.columns[[1] + list(range(6, df_tor_final.shape[1]))]]

,Borough,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
2,Scarborough,Pizza Place,Fast Food Restaurant,Bank,Coffee Shop,Sandwich Place
4,Scarborough,Coffee Shop,Gas Station,Pharmacy,Indian Restaurant,Bank
6,Scarborough,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Grocery Store
10,Scarborough,Coffee Shop,Asian Restaurant,Electronics Store,Restaurant,Fast Food Restaurant
11,Scarborough,Middle Eastern Restaurant,Pizza Place,Grocery Store,Burger Joint,Smoke Shop
12,Scarborough,Chinese Restaurant,Shopping Mall,Pizza Place,Bakery,Sandwich Place
13,Scarborough,Pharmacy,Coffee Shop,Fast Food Restaurant,Bank,Park
14,Scarborough,Chinese Restaurant,Bakery,Korean Restaurant,Pizza Place,Fast Food Restaurant
15,Scarborough,Chinese Restaurant,Grocery Store,Bank,Bakery,Coffee Shop
17,North York,Coffee Shop,Park,Pharmacy,Bank,Residential Building (Apartment / Condo)


In [70]:
df_tor_final.loc[df_tor_final['Labels'] == 1, df_tor_final.columns[[1] + list(range(6, df_tor_final.shape[1]))]]

,Borough,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
20,North York,Park,Pool,Fast Food Restaurant,Eastern European Restaurant,Electronics Store


In [71]:
df_tor_final.loc[df_tor_final['Labels'] == 2, df_tor_final.columns[[1] + list(range(6, df_tor_final.shape[1]))]]

,Borough,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
0,Scarborough,Coffee Shop,Fast Food Restaurant,Trail,Restaurant,Spa
5,Scarborough,Ice Cream Shop,Sandwich Place,Bowling Alley,Train Station,Japanese Restaurant
18,North York,Coffee Shop,Clothing Store,Sandwich Place,Restaurant,Bakery
21,North York,Korean Restaurant,Café,Pizza Place,Middle Eastern Restaurant,Coffee Shop
22,North York,Coffee Shop,Pizza Place,Bubble Tea Shop,Ramen Restaurant,Korean Restaurant
26,North York,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Burger Joint
27,North York,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Burger Joint
28,North York,Coffee Shop,Bank,Pizza Place,Gas Station,Supermarket
29,North York,Furniture / Home Store,Pizza Place,Coffee Shop,Restaurant,Sushi Restaurant
34,North York,Coffee Shop,Men's Store,Grocery Store,Portuguese Restaurant,Sporting Goods Shop


In [72]:
df_tor_final.loc[df_tor_final['Labels'] == 3, df_tor_final.columns[[1] + list(range(6, df_tor_final.shape[1]))]]

,Borough,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
102,Etobicoke,Dog Run,Rental Car Location,Lounge,Coffee Shop,Zoo


In [73]:
df_tor_final.loc[df_tor_final['Labels'] == 4, df_tor_final.columns[[1] + list(range(6, df_tor_final.shape[1]))]]

,Borough,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
3,Scarborough,Park,Coffee Shop,Fast Food Restaurant,Pharmacy,Mobile Phone Shop
7,Scarborough,Intersection,Pharmacy,Park,Coffee Shop,Bakery
8,Scarborough,Pizza Place,Ice Cream Shop,Beach,Sports Bar,Park
9,Scarborough,Park,Restaurant,Thai Restaurant,Diner,Gym
23,North York,Park,Restaurant,Tennis Court,Coffee Shop,Convenience Store
25,North York,Park,Convenience Store,Bus Stop,Pharmacy,Shopping Mall
35,East York,Gym / Fitness Center,Pizza Place,Brewery,Bank,Pharmacy
36,East York,Coffee Shop,Park,Thai Restaurant,Pizza Place,Sandwich Place
44,Central Toronto,Park,Bookstore,Gym / Fitness Center,Trail,Café
50,Downtown Toronto,Coffee Shop,Park,Grocery Store,Breakfast Spot,Juice Bar


In [74]:
df_tor_final.loc[df_tor_final['Labels'] == 5, df_tor_final.columns[[1] + list(range(6, df_tor_final.shape[1]))]]

,Borough,Top Venue Number 1,Top Venue Number 2,Top Venue Number 3,Top Venue Number 4,Top Venue Number 5
1,Scarborough,Breakfast Spot,Burger Joint,Park,Playground,Italian Restaurant
